In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Cursuri/Business Analyst/Business Analyst course/Segmentation/RFM

/content/drive/MyDrive/Cursuri/Business Analyst/Business Analyst course/Segmentation/RFM


In [15]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta

In [8]:
data = pd.read_csv("online_shoppers.csv",
                   encoding = "ISO-8859-1").dropna()
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [9]:
data["Description"].value_counts()

WHITE HANGING HEART T-LIGHT HOLDER    2070
REGENCY CAKESTAND 3 TIER              1905
JUMBO BAG RED RETROSPOT               1662
ASSORTED COLOUR BIRD ORNAMENT         1418
PARTY BUNTING                         1416
                                      ... 
ANTIQUE RASPBERRY FLOWER EARRINGS        1
WALL ART,ONLY ONE PERSON                 1
GOLD/AMBER DROP EARRINGS W LEAF          1
INCENSE BAZAAR PEACH                     1
PINK BAROQUE FLOCK CANDLE HOLDER         1
Name: Description, Length: 3896, dtype: int64

#Data Preparation

In [10]:
#Creating Sales Column
data['sales'] = data['Quantity'] * data['UnitPrice']
data.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.3


In [16]:
#Dates - Transform Date Variable
data['InvoiceDate'] = pd.to_datetime(data["InvoiceDate"])
data['InvoiceDate']

#Get last data available
snapshot_date = data['InvoiceDate'].max() + timedelta(days = 1)
snapshot_date

Timestamp('2011-12-10 12:50:00')

In [19]:
#Aggregate on Customer Level
df = data.groupby(['CustomerID']).agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days, #recency - from last date drop the last date of the invoice of the customer
    'InvoiceNo': 'count',
    'sales': 'sum'
    })

df.head()

,InvoiceDate,InvoiceNo,sales
CustomerID,,,
12346.0,326,2,0.00
12347.0,2,182,4310.00
12348.0,75,31,1797.24
12349.0,19,73,1757.55
12350.0,310,17,334.40


In [22]:
#Create basket / monetary variable
df["Monetary"] = df['sales'] / df['InvoiceNo']
df.head(2)

,InvoiceDate,InvoiceNo,sales,Monetary
CustomerID,,,,
12346.0,326,2,0.0,0.000000
12347.0,2,182,4310.0,23.681319


In [30]:
#Cleaning data

# 1. Dropping Sales Variable
df.head(1)

,InvoiceDate,InvoiceNo,Monetary
CustomerID,,,
12346.0,326,2,0.0


In [31]:
#Changing variables names
df.rename(columns = {
    'InvoiceDate': 'Recency',
    'InvoiceNo': 'Frequency',
}, inplace = True)

df.head(1)

,Recency,Frequency,Monetary
CustomerID,,,
12346.0,326,2,0.0


#RFM Preparation

In [32]:
#Frequency Groups
df['F'] = pd.qcut(x = df['Frequency'],
                  q = 4,
                  labels = range(1, 5, 1))
df.head()

,Recency,Frequency,Monetary,F
CustomerID,,,,
12346.0,326,2,0.000000,1
12347.0,2,182,23.681319,4
12348.0,75,31,57.975484,2
12349.0,19,73,24.076027,3
12350.0,310,17,19.670588,1


In [33]:
df['F']

CustomerID
12346.0    1
12347.0    4
12348.0    2
12349.0    3
12350.0    1
          ..
18280.0    1
18281.0    1
18282.0    1
18283.0    4
18287.0    3
Name: F, Length: 4372, dtype: category
Categories (4, int64): [1 < 2 < 3 < 4]

In [34]:
df['M'] = pd.qcut(x = df['Monetary'],
                  q = 4,
                  labels = range(1,5,1))

df['R'] = pd.qcut(x = df['Recency'],
                  q = 4,
                  labels = range(1,5,1))

df.head()

,Recency,Frequency,Monetary,F,M,R
CustomerID,,,,,,
12346.0,326,2,0.000000,1,1,4
12347.0,2,182,23.681319,4,4,1
12348.0,75,31,57.975484,2,4,3
12349.0,19,73,24.076027,3,4,2
12350.0,310,17,19.670588,1,3,4


In [35]:
#RFM Score
df['RFM'] = df[['R', 'F', 'M']].sum(axis = 1)
df.head()

,Recency,Frequency,Monetary,F,M,R,RFM
CustomerID,,,,,,,
12346.0,326,2,0.000000,1,1,4,6.0
12347.0,2,182,23.681319,4,4,1,9.0
12348.0,75,31,57.975484,2,4,3,9.0
12349.0,19,73,24.076027,3,4,2,9.0
12350.0,310,17,19.670588,1,3,4,8.0


In [36]:
#RFM Function
def rfm_segment(df):
  if df['RFM'] >= 11:
    return 'Supper Score'
  elif df['RFM'] < 11 and df['RFM'] >= 8:
    return 'Future Champion'
  elif df['RFM'] >= 6 and df['RFM'] < 8:
    return 'High Potentional'
  else:
    return "Low Relevance"

RFM

In [38]:
#Apply RFM interpretation function

df["RFM_level"] = df.apply(rfm_segment, axis = 1)
df.head()
df["RFM_level"].value_counts()

Future Champion     2136
High Potentional    1880
Low Relevance        315
Supper Score          41
Name: RFM_level, dtype: int64

In [40]:
#Looking into the segments
df.groupby('RFM_level').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': ['mean', 'count']
})

Recency   Frequency   Monetary      
                        mean        mean       mean count
RFM_level                                                
Future Champion   120.406367   90.010768  40.971031  2136
High Potentional   70.129255  106.175000  18.252143  1880
Low Relevance      21.860317   31.984127   8.763361   315
Supper Score      158.853659  119.073171  36.476997    41